In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import datetime


In [7]:
#Import Data
(ds_train, ds_test), metadata = tfds.load(
    'stanford_dogs',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    as_supervised=True,
)

Shuffling and writing examples to /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete9E9NH6/stanford_dogs-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete9E9NH6/stanford_dogs-test.tfrecord


Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [8]:
#The test set is pretty large, we merge some of it into the train set
ds_test = ds_test.shuffle(int(tf.data.experimental.cardinality(ds_test)), reshuffle_each_iteration=False)
ds_train = ds_train.concatenate(ds_test.take(5000))
ds_test = ds_test.skip(5000)

In [9]:
#Set Parameters
NUM_CLASSES = metadata.features['label'].num_classes
IMG_SIZE = 224
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

NUM_TRAINING = int(tf.data.experimental.cardinality(ds_train))
NUM_TEST = int(tf.data.experimental.cardinality(ds_test))

In [11]:
#Preprocess the data
train = (ds_train
         .map(util.preprocess, num_parallel_calls=AUTOTUNE)
         .map(util.augment, num_parallel_calls=AUTOTUNE)
         .map(
             lambda image, label: (tf.keras.applications.resnet50.preprocess_input(image), label),
             num_parallel_calls=AUTOTUNE)
         .batch(64, drop_remainder=True)
         .prefetch(AUTOTUNE)
         .repeat()
        )
 
val = (ds_test
       .map(util.preprocess, num_parallel_calls=AUTOTUNE)
       .map(
             lambda image, label: (tf.keras.applications.resnet50.preprocess_input(image), label),
             num_parallel_calls=AUTOTUNE)
       .cache()
       .batch(64, drop_remainder=True)
      )

In [12]:
#Create first model for finetuning
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False


model = keras.Sequential([
        keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])


callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
]


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.01, decay_steps=50, decay_rate=0.96, staircase=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(train, 
          epochs=5, 
          callbacks=callbacks, 
          validation_data=val, 
          steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
          validation_steps=NUM_TEST//BATCH_SIZE
         )

model.save('model_1')

94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/5
265/265 [==============================] - 93s 351ms/step - loss: 3.7867 - accuracy: 0.5226 - val_loss: 1.7368 - val_accuracy: 0.7571
Epoch 2/5
265/265 [==============================] - 90s 339ms/step - loss: 2.6325 - accuracy: 0.6874 - val_loss: 1.3057 - val_accuracy: 0.8088
Epoch 3/5
265/265 [==============================] - 90s 341ms/step - loss: 2.2208 - accuracy: 0.7312 - val_loss: 1.1695 - val_accuracy: 0.8230
Epoch 4/5
265/265 [==============================] - 90s 339ms/step - loss: 1.8853 - accuracy: 0.7595 - val_loss: 0.8146 - val_accuracy: 0.8668
Epoch 5/5
265/265 [==============================] - 89s 337ms/step - loss: 1.4995 - accuracy: 0.7968 - val_loss: 0.7571 - val_accuracy: 0.8753


In [14]:
base_model = keras.models.load_model('model_1', compile=False)

for layer in base_model.layers:
    layer.trainable = True
    
for layer in base_model.layers[:10]:
    layer.trainable = False
    
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.001, decay_steps=50, decay_rate=0.96, staircase=True
)
    
    
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
]

base_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

base_model.fit(train, 
          epochs=5, 
          callbacks=callbacks, 
          validation_data=val, 
          steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
          validation_steps=NUM_TEST//BATCH_SIZE
         )

base_model.save('model_2')

Epoch 1/5
265/265 [==============================] - 93s 351ms/step - loss: 1.3801 - accuracy: 0.8047 - val_loss: 0.7775 - val_accuracy: 0.8676
Epoch 2/5
265/265 [==============================] - 91s 342ms/step - loss: 1.3598 - accuracy: 0.8078 - val_loss: 0.7120 - val_accuracy: 0.8710
Epoch 3/5
265/265 [==============================] - 89s 338ms/step - loss: 1.3282 - accuracy: 0.8120 - val_loss: 0.6666 - val_accuracy: 0.8858
Epoch 4/5
265/265 [==============================] - 92s 346ms/step - loss: 1.3591 - accuracy: 0.8088 - val_loss: 0.6823 - val_accuracy: 0.8813
Epoch 5/5
265/265 [==============================] - 91s 342ms/step - loss: 1.3721 - accuracy: 0.8066 - val_loss: 0.7161 - val_accuracy: 0.8813
